In [2]:
import pandas as pd 
import numpy as np
import os
import torch
from torch.utils.data import DataLoader, Dataset

In [333]:
df_even = pd.DataFrame()
df_odd = pd.DataFrame()
for i in sorted(os.listdir('eeg_emotion_6class/odd')):
    if '.csv' in i:
        read = pd.read_csv(f'eeg_emotion_6class/odd/{i}')
        read = read[['delta', 'lowAlpha', 'highAlpha', 'lowBeta', 'highBeta',
       'lowGamma', 'middleGamma', 'theta', 'pid',
       'trans_time', 'new_trans_time', 'seconds', 'arousal', 'valence',
       'happy', 'angry', 'nervous', 'sad']]
        
        df_odd = pd.concat([df_odd, read], axis = 0, ignore_index = True)

        
for i in sorted(os.listdir('eeg_emotion_6class/even')):
    if '.csv' in i:
        read = pd.read_csv(f'eeg_emotion_6class/even/{i}')
        read = read[['delta', 'lowAlpha', 'highAlpha', 'lowBeta', 'highBeta',
       'lowGamma', 'middleGamma', 'theta', 'pid',
       'trans_time', 'new_trans_time', 'seconds', 'arousal', 'valence',
       'happy', 'angry', 'nervous', 'sad']]
        
        df_even = pd.concat([df_even, read], axis = 0, ignore_index = True)
        
df_even.to_csv('df_even_eeg.csv')
df_odd.to_csv('df_odd_eeg.csv')
            

                       

In [85]:
columns = ['delta', 'lowAlpha', 'highAlpha', 'lowBeta', 'highBeta',
       'lowGamma', 'middleGamma', 'theta']

even = pd.read_csv('df_even_eeg.csv')
odd = pd.read_csv('df_odd_eeg.csv')

mean = even[columns].mean()

std = even[columns].std()

for i in range(len(even)):
    
    even.iloc[i, 1] = ((even.iloc[i, 1] - mean[0]) / std[0]).round(3) # delta
    
    even.iloc[i, 2] = ((even.iloc[i, 2] - mean[1]) / std[1]).round(3) # lowAlpha
    
    even.iloc[i, 3] = ((even.iloc[i, 3] - mean[2]) / std[2]).round(3) # highAlpha
    
    even.iloc[i, 4] = ((even.iloc[i, 4] - mean[3]) / std[3]).round(3) # lowBeta
    
    even.iloc[i, 5] = ((even.iloc[i, 5] - mean[4]) / std[4]).round(3) # highBeta
    
    even.iloc[i, 6] = ((even.iloc[i, 6] - mean[5]) / std[5]).round(3) # lowGamma
    
    even.iloc[i, 7] = ((even.iloc[i, 7] - mean[6]) / std[6]).round(3) # middleGamma
    
    even.iloc[i, 8] = ((even.iloc[i, 8] - mean[7]) / std[7]).round(3) # theta
    

even.angry = even.angry.round().astype(int)
even.nervous = even.nervous.round().astype(int)
even.sad = even.sad.round().astype(int)
even.happy = even.happy.round().astype(int)
even.arousal = even.arousal.round().astype(int)
even.valence = even.valence.round().astype(int)

even.to_csv('df_even_norm_eeg.csv')

mean = odd[columns].mean()

std = odd[columns].std()

for i in range(len(even)):
    
    odd.iloc[i, 1] = ((odd.iloc[i, 1] - mean[0]) / std[0]).round(3) # delta
    
    odd.iloc[i, 2] = ((odd.iloc[i, 2] - mean[1]) / std[1]).round(3) # lowAlpha
    
    odd.iloc[i, 3] = ((odd.iloc[i, 3] - mean[2]) / std[2]).round(3) # highAlpha
    
    odd.iloc[i, 4] = ((odd.iloc[i, 4] - mean[3]) / std[3]).round(3) # lowBeta
    
    odd.iloc[i, 5] = ((odd.iloc[i, 5] - mean[4]) / std[4]).round(3) # highBeta
    
    odd.iloc[i, 6] = ((odd.iloc[i, 6] - mean[5]) / std[5]).round(3) # lowGamma
    
    odd.iloc[i, 7] = ((odd.iloc[i, 7] - mean[6]) / std[6]).round(3) # middleGamma
    
    odd.iloc[i, 8] = ((odd.iloc[i, 8] - mean[7]) / std[7]).round(3) # theta


odd.angry = odd.angry.round().astype(int)
odd.nervous = odd.nervous.round().astype(int)
odd.sad = odd.sad.round().astype(int)
odd.happy = odd.happy.round().astype(int)
odd.arousal = odd.arousal.round().astype(int)
odd.valence = odd.valence.round().astype(int)
    
odd.to_csv('df_odd_norm_eeg.csv')


odd.to_csv('df_odd_norm_eeg.csv')

In [27]:
df_even = pd.read_csv('df_even_norm_eeg.csv')
df_odd = pd.read_csv('df_odd_norm_eeg.csv')
emotion = 'angry'
eeg = [emotion, 'delta', 'lowAlpha', 'highAlpha',
       'lowBeta', 'highBeta', 'lowGamma', 'middleGamma', 'theta']

even_id = [emotion, 'delta', 'lowAlpha', 'highAlpha',
       'lowBeta', 'highBeta', 'lowGamma', 'middleGamma', 'theta', 'seconds']

odd_id = [emotion, 'delta', 'lowAlpha', 'highAlpha',
       'lowBeta', 'highBeta', 'lowGamma', 'middleGamma', 'theta', 'seconds']

even_id = df_even[even_id]#.values
odd_id = df_odd[odd_id]#.values

even = df_even[eeg]#.values
odd = df_odd[eeg]#.values

class EEGDataset(Dataset):
    def __init__(self, signal, label):

        self._signal = torch.FloatTensor(signal)
        self._label = torch.LongTensor(label)
        
    @property
    def n_insts(self):
        ''' Property for dataset size '''
        return len(self._label)

    @property
    def sig_len(self):
        return self._signal.shape[1]

    def __len__(self):
        return self.n_insts
    
    def __getitem__(self,idx):
                                     
        return self._signal[idx], self._label[idx]

In [359]:
for p in range(11, 32, 2): 

    read= pd.read_csv(f'eeg_emotion_6class/odd/0{p}_eeg.csv')
    read2 = pd.read_csv(f'eeg_emotion_6class/odd/0{p}_eeg.csv')
    read2 = read2.drop_duplicates(subset=['seconds'], keep='first')
    read2 = read2[['pid', 'trans_time', 'seconds', 'arousal', 'valence',
    'happy', 'angry', 'nervous', 'sad']].reset_index()

    read = read.astype({'delta': str,
                    'lowAlpha' : str,
                    'highAlpha' : str,
                    'lowBeta' : str,
                    'highBeta' : str,
                    'lowGamma' : str,
                    'middleGamma' : str,
                    'theta' : str
                             })


    x = read.groupby('seconds')[['delta', 'lowAlpha', 'highAlpha', 'lowBeta', 'highBeta',
       'lowGamma', 'middleGamma', 'theta']].agg(','.join).reset_index()

    delta = x['delta'].str.split(',', expand=True)
    delta = delta.rename(columns = {0: 'delta0',
                                1: 'delta1',
                                2: 'delta2',
                                3: 'delta3',
                                4: 'delta4',
                                5: 'delta5',
                                6: 'delta6'})
    lowAlpha = x['lowAlpha'].str.split(',', expand=True)
    lowAlpha = lowAlpha.rename(columns = {0: 'lowAlpha0',
                                1: 'lowAlpha1',
                                2: 'lowAlpha2',
                                3: 'lowAlpha3',
                                4: 'lowAlpha4',
                                5: 'lowAlpha5',
                                6: 'lowAlpha6'})
    highAlpha = x['highAlpha'].str.split(',', expand=True)
    highAlpha = highAlpha.rename(columns = {0: 'highAlpha0',
                                1: 'highAlpha1',
                                2: 'highAlpha2',
                                3: 'highAlpha3',
                                4: 'highAlpha4',
                                5: 'highAlpha5',
                                6: 'highAlpha6'})
    lowBeta = x['lowBeta'].str.split(',', expand=True)
    lowBeta = lowBeta.rename(columns = {0: 'lowBeta0',
                                1: 'lowBeta1',
                                2: 'lowBeta2',
                                3: 'lowBeta3',
                                4: 'lowBeta4',
                                5: 'lowBeta5',
                                6: 'lowBeta6'})
    highBeta = x['highBeta'].str.split(',', expand=True)
    highBeta = highBeta.rename(columns = {0: 'highBeta0',
                                1: 'highBeta1',
                                2: 'highBeta2',
                                3: 'highBeta3',
                                4: 'highBeta4',
                                5: 'highBeta5',
                                6: 'highBeta6'})
    lowGamma = x['lowGamma'].str.split(',', expand=True)
    lowGamma = lowGamma.rename(columns = {0: 'lowGamma0',
                                1: 'lowGamma1',
                                2: 'lowGamma2',
                                3: 'lowGamma3',
                                4: 'lowGamma4',
                                5: 'lowGamma5',
                                6: 'lowGamma6'})
    middleGamma = x['middleGamma'].str.split(',', expand=True)
    middleGamma = middleGamma.rename(columns = {0: 'middleGamma0',
                                1: 'middleGamma1',
                                2: 'middleGamma2',
                                3: 'middleGamma3',
                                4: 'middleGamma4',
                                5: 'middleGamma5',
                                6: 'middleGamma6'})
    theta = x['theta'].str.split(',', expand=True)
    theta = theta.rename(columns = {0: 'theta0',
                                1: 'theta1',
                                2: 'theta2',
                                3: 'theta3',
                                4: 'theta4',
                                5: 'theta5',
                                6: 'theta6'})
    df = pd.concat([delta, lowAlpha, highAlpha, lowBeta, highBeta, lowGamma, middleGamma, theta, read2], axis= 1)

    columns = ['delta0', 'delta1', 'delta2', 'delta3', 'delta4', 'delta5', 
           'lowAlpha0', 'lowAlpha1', 'lowAlpha2', 'lowAlpha3', 'lowAlpha4', 'lowAlpha5',
           'highAlpha0', 'highAlpha1', 'highAlpha2', 'highAlpha3', 'highAlpha4', 'highAlpha5', 
           'lowBeta0', 'lowBeta1', 'lowBeta2', 'lowBeta3', 'lowBeta4', 'lowBeta5', 
           'highBeta0', 'highBeta1', 'highBeta2', 'highBeta3', 'highBeta4', 'highBeta5', 
           'lowGamma0', 'lowGamma1', 'lowGamma2', 'lowGamma3', 'lowGamma4', 'lowGamma5', 
           'middleGamma0', 'middleGamma1', 'middleGamma2', 'middleGamma3', 'middleGamma4', 'middleGamma5',
           'theta0', 'theta1', 'theta2', 'theta3', 'theta4', 'theta5']

    df[columns] = df[columns].apply(pd.to_numeric)

    mean = df[columns].mean()

    std = df[columns].std()

    for i in range(len(df)):

        df.iloc[i, 0] = ((df.iloc[i, 0] - mean[0]) / std[0]).round(3) # delta

        df.iloc[i, 1] = ((df.iloc[i, 1] - mean[1]) / std[1]).round(3) 

        df.iloc[i, 2] = ((df.iloc[i, 2] - mean[2]) / std[2]).round(3) 

        df.iloc[i, 3] = ((df.iloc[i, 3] - mean[3]) / std[3]).round(3) 

        df.iloc[i, 4] = ((df.iloc[i, 4] - mean[4]) / std[4]).round(3) 

        df.iloc[i, 5] = ((df.iloc[i, 5] - mean[5]) / std[5]).round(3) 

        df.iloc[i, 7] = ((df.iloc[i, 7] - mean[6]) / std[6]).round(3) # lowAlpha

        df.iloc[i, 8] = ((df.iloc[i, 8] - mean[7]) / std[7]).round(3)

        df.iloc[i, 9] = ((df.iloc[i, 9] - mean[8]) / std[8]).round(3)

        df.iloc[i, 10]= ((df.iloc[i, 10] - mean[9]) / std[9]).round(3)

        df.iloc[i, 11] = ((df.iloc[i, 11] - mean[10]) / std[10]).round(3)

        df.iloc[i, 12] = ((df.iloc[i, 12] - mean[11]) / std[11]).round(3)

        df.iloc[i, 14] = ((df.iloc[i, 14] - mean[12]) / std[12]).round(3) # highAlpha

        df.iloc[i, 15]  = ((df.iloc[i, 15] - mean[13]) / std[13]).round(3)

        df.iloc[i, 16] = ((df.iloc[i, 16] - mean[14]) / std[14]).round(3)

        df.iloc[i, 17] = ((df.iloc[i, 17] - mean[15]) / std[15]).round(3)

        df.iloc[i, 18] = ((df.iloc[i, 18] - mean[16]) / std[16]).round(3)

        df.iloc[i, 19] = ((df.iloc[i, 19] - mean[17]) / std[17]).round(3)

        df.iloc[i, 21] = ((df.iloc[i, 21] - mean[18]) / std[18]).round(3) # lowBeta

        df.iloc[i, 22] = ((df.iloc[i, 22] - mean[19]) / std[19]).round(3)

        df.iloc[i, 23] = ((df.iloc[i, 23] - mean[20]) / std[20]).round(3)

        df.iloc[i, 24] = ((df.iloc[i, 24] - mean[21]) / std[21]).round(3)

        df.iloc[i, 25] = ((df.iloc[i, 25] - mean[22]) / std[22]).round(3)

        df.iloc[i, 26] = ((df.iloc[i, 26] - mean[23]) / std[23]).round(3)

        df.iloc[i, 28] = ((df.iloc[i, 28] - mean[24]) / std[24]).round(3) # highBeta

        df.iloc[i, 29] = ((df.iloc[i, 29] - mean[25]) / std[25]).round(3)

        df.iloc[i, 30] = ((df.iloc[i, 30] - mean[26]) / std[26]).round(3)

        df.iloc[i, 31] = ((df.iloc[i, 31] - mean[27]) / std[27]).round(3)

        df.iloc[i, 32] = ((df.iloc[i, 32] - mean[28]) / std[28]).round(3)

        df.iloc[i, 33] = ((df.iloc[i, 33] - mean[29]) / std[29]).round(3)

        df.iloc[i, 35] = ((df.iloc[i, 35] - mean[30]) / std[30]).round(3) # lowGamma

        df.iloc[i, 36] = ((df.iloc[i, 36] - mean[31]) / std[31]).round(3)

        df.iloc[i, 37] = ((df.iloc[i, 37] - mean[32]) / std[32]).round(3)

        df.iloc[i, 38] = ((df.iloc[i, 38] - mean[33]) / std[33]).round(3)

        df.iloc[i, 39] = ((df.iloc[i, 39] - mean[34]) / std[34]).round(3)

        df.iloc[i, 40] = ((df.iloc[i, 40] - mean[35]) / std[35]).round(3)

        df.iloc[i, 42] = ((df.iloc[i, 42] - mean[36]) / std[36]).round(3) #middleGamma

        df.iloc[i, 43] = ((df.iloc[i, 43] - mean[37]) / std[37]).round(3)

        df.iloc[i, 44] = ((df.iloc[i, 44] - mean[38]) / std[38]).round(3)

        df.iloc[i, 45] = ((df.iloc[i, 45] - mean[39]) / std[39]).round(3)

        df.iloc[i, 46] = ((df.iloc[i, 46] - mean[40]) / std[40]).round(3)

        df.iloc[i, 47] = ((df.iloc[i, 47] - mean[41]) / std[41]).round(3)

        df.iloc[i, 49] = ((df.iloc[i, 49] - mean[42]) / std[42]).round(3) # theta

        df.iloc[i, 50] = ((df.iloc[i, 50] - mean[43]) / std[43]).round(3)

        df.iloc[i, 51] = ((df.iloc[i, 51] - mean[44]) / std[44]).round(3)

        df.iloc[i, 52] = ((df.iloc[i, 52] - mean[45]) / std[45]).round(3)

        df.iloc[i, 53] = ((df.iloc[i, 53] - mean[46]) / std[46]).round(3)

        df.iloc[i, 54] = ((df.iloc[i, 54] - mean[47]) / std[47]).round(3)

    df.angry = df.angry.round().astype(int)
    df.nervous = df.nervous.round().astype(int)
    df.sad = df.sad.round().astype(int)
    df.happy = df.happy.round().astype(int)
    df.arousal = df.arousal.round().astype(int)
    df.valence = df.valence.round().astype(int)

    df = df[['delta0', 'lowAlpha0', 'highAlpha0','lowBeta0','highBeta0', 'lowGamma0', 'middleGamma0', 'theta0',
         'delta1', 'lowAlpha1', 'highAlpha1', 'lowBeta1', 'highBeta1', 'lowGamma1', 'middleGamma1', 'theta1',
         'delta2', 'lowAlpha2', 'highAlpha2', 'lowBeta2', 'highBeta2', 'lowGamma2', 'middleGamma2', 'theta2',
         'delta3', 'lowAlpha3', 'highAlpha3', 'lowBeta3', 'highBeta3', 'lowGamma3', 'middleGamma3', 'theta3',
         'delta4', 'lowAlpha4', 'highAlpha4', 'lowBeta4', 'highBeta4', 'lowGamma4', 'middleGamma4', 'theta4',
         'delta5', 'lowAlpha5', 'highAlpha5', 'lowBeta5', 'highBeta5', 'lowGamma5', 'middleGamma5', 'theta5',
         'pid', 'trans_time', 'seconds', 'arousal', 'valence', 'happy', 'angry', 'nervous', 'sad']]
    
    df = df.fillna(0)

    df.to_csv(f'eeg_long/odd/0{p}_eeg_norm.csv')

In [310]:
def to_emotion(rating):
  rating = int(rating)
  if rating == 2:
    return 1
  elif rating == 4:
    return 3
  elif rating == 3:
    return 2
  else: 
    return 0

In [364]:
new_df = pd.DataFrame()
new_df2 = pd.DataFrame()
for i in sorted(os.listdir('eeg_long/odd')):
    if '.csv' in i:
       
        read1 = pd.read_csv(f'eeg_long/odd/{i}')
    
        new_df = pd.concat([new_df, read1], axis=0)

new_df = new_df.reset_index()

for i in sorted(os.listdir('eeg_long/even')):
    if '.csv' in i:
        # print(i)
        read2 = pd.read_csv(f'eeg_long/even/{i}')
        new_df2 = pd.concat([new_df2, read2], axis=0)

new_df2 = new_df2.reset_index()

new_df = pd.concat([new_df, new_df2], axis = 1)

# new_df['arousal'] = new_df.arousal.apply(to_emotion)
# new_df['arousal'] = new_df.valence2_trans.apply(to_emotion)
# new_df['arousal'] = new_df.valence2_trans.apply(to_emotion)
# new_df['arousal'] = new_df.valence2_trans.apply(to_emotion)
# new_df['arousal'] = new_df.valence2_trans.apply(to_emotion)
# new_df['arousal'] = new_df.valence2_trans.apply(to_emotion)
# new_df['arousal'] = new_df.valence2_trans.apply(to_emotion)


new_df.to_csv('df.csv')
    

In [365]:
df = pd.read_csv('df.csv')


False


In [24]:
df_eeg = pd.read_csv('main_trans/df_eeg.csv')
df = pd.read_csv('main_trans/df.csv')

for i in range(len(df)):
    if df.iloc[i, 0] != df_eeg.iloc[i,0]:
        print(df.iloc[i,0])
        print(df_eeg.iloc[i, 0])
        print()


In [26]:
df_eeg.columns

Index(['Unnamed: 0', 'delta0', 'lowAlpha0', 'highAlpha0', 'lowBeta0',
       'highBeta0', 'lowGamma0', 'middleGamma0', 'theta0', 'delta1',
       ...
       'theta5_2', 'pid_2', 'trans_time_2', 'seconds_2', 'arousal_2',
       'valence_2', 'happy_2', 'angry_2', 'nervous_2', 'sad_2'],
      dtype='object', length=115)

In [ ]:
columns = ['delta0', 'delta1', 'delta2', 'delta3', 'delta4', 'delta5', 
           'lowAlpha0', 'lowAlpha1', 'lowAlpha2', 'lowAlpha3', 'lowAlpha4', 'lowAlpha5',
           'highAlpha0', 'highAlpha1', 'highAlpha2', 'highAlpha3', 'highAlpha4', 'highAlpha5', 
           'lowBeta0', 'lowBeta1', 'lowBeta2', 'lowBeta3', 'lowBeta4', 'lowBeta5', 
           'highBeta0', 'highBeta1', 'highBeta2', 'highBeta3', 'highBeta4', 'highBeta5', 
           'lowGamma0', 'lowGamma1', 'lowGamma2', 'lowGamma3', 'lowGamma4', 'lowGamma5', 
           'middleGamma0', 'middleGamma1', 'middleGamma2', 'middleGamma3', 'middleGamma4', 'middleGamma5',
           'theta0', 'theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'arousal_2',
       'valence_2', 'happy_2', 'angry_2', 'nervous_2', 'sad_2']


df = df['new_words']

df = pd.concat([df, df_eg





In [360]:
columns = ['delta0', 'delta1', 'delta2', 'delta3', 'delta4', 'delta5', 
           'lowAlpha0', 'lowAlpha1', 'lowAlpha2', 'lowAlpha3', 'lowAlpha4', 'lowAlpha5',
           'highAlpha0', 'highAlpha1', 'highAlpha2', 'highAlpha3', 'highAlpha4', 'highAlpha5', 
           'lowBeta0', 'lowBeta1', 'lowBeta2', 'lowBeta3', 'lowBeta4', 'lowBeta5', 
           'highBeta0', 'highBeta1', 'highBeta2', 'highBeta3', 'highBeta4', 'highBeta5', 
           'lowGamma0', 'lowGamma1', 'lowGamma2', 'lowGamma3', 'lowGamma4', 'lowGamma5', 
           'middleGamma0', 'middleGamma1', 'middleGamma2', 'middleGamma3', 'middleGamma4', 'middleGamma5',
           'theta0', 'theta1', 'theta2', 'theta3', 'theta4', 'theta5']

for i in sorted(os.listdir('eeg_long/odd')):
    if '.csv' in i:
        read1= pd.read_csv(f'eeg_long/odd/{i}')
        print(read1[columns].isnull().values.any())
            
print()
for i in sorted(os.listdir('eeg_long/even')):
    if '.csv' in i:
        read1= pd.read_csv(f'eeg_long/even/{i}')
        
        print(read1[columns].isnull().values.any())
           


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [ ]:
bool_series = pd.isnull(data["Gender"])